<a href="https://colab.research.google.com/github/sreeduttsasikumar/Coursera_Capstone/blob/main/Toronto_Neighbourhood_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Toronto Neighbourhood Clustering Project<center><h1>

Import the libraries needed for the project

In [109]:
import pandas as pd
import numpy as np

By using panda library scrape the wikipedia page and fetch the Toronto neighbourhood table and then convert to a panda dataframe

In [110]:
#Assign the wikipedia url to variable url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#fetch from the url using panda function
data_from_url = pd.read_html(url)
#get the first table which is the relevant table for our project
data_from_url = data_from_url[0]
#create a dataframe from the data fetched from url
toronto_df = data_from_url[['Postal Code', 'Borough', 'Neighbourhood']]

Cleaning the data

In [111]:
#drop those rows where Borough has value "Not Assigned"
toronto_df.drop(toronto_df[toronto_df['Borough'] == 'Not assigned'].index, inplace= True)
#reset the index value
toronto_df.reset_index(inplace= True, drop=True)
#replacing Neighbourhood column with Borough if Neighbourhood = Not Assigned
toronto_df['Neighbourhood'] = np.where(toronto_df['Neighbourhood'] == 'Not assigned', toronto_df['Borough'], toronto_df['Neighbourhood'])
print("Number of rows in the cleaned dataset is",toronto_df.shape[0])

Number of rows in the cleaned dataset is 103


From Googles Geocoder API we need find latitude and longitudes of all available postal codes in the dataset. Also need to update those values in the main data set

In [112]:
#import required package
!pip install geocoder
import geocoder

Tried below logic for fetching cordinates, but not getting any results. Hencing going to use the data provided in the assignment

In [113]:
#for postcode in toronto_df['Postal Code']:
#  # initialize your variable to None
#  lat_lng_coords = None
#  # loop until you get the coordinates
#  while(lat_lng_coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
#    lat_lng_coords = g.latlng
#  latitude = lat_lng_coords[0]
#  longitude = lat_lng_coords[1]
#  print("PostCode-",postcode,"Latitude-",latitude,"Longitude-",longitude)

In [114]:
geocoder.google('M2L, Toronto, Ontario')

<[REQUEST_DENIED] Google - Geocode [empty]>

Use the csv shared in the assignment, corresponding lattitudes and longitudes were assigned against the postal codes

In [115]:
#fetch the csv file and assign it to a dataframe
pc_lat_long = pd.read_csv("http://cocl.us/Geospatial_data")
#using pandas merge function merge the initial data set and the coordinte data set using the common column Postal Code
toronto_df = pd.merge(toronto_df, pc_lat_long, left_on='Postal Code', right_on='Postal Code')
toronto_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
